In [3]:
import numpy as np

from cb25d.batch import run_batch_simulation
from cb25d.compare_gamma_original import run_gamma_comparison_original
from cb25d.interactive import run_interactive_simulation
from cb25d.notebook import init
from cb25d.simulation_impl_original import (
    SimulationImplOriginal,
    SimulationRecorderOriginal,
    SimulationRendererOriginal,
    generate_initial_conditions,
)

init()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
run_interactive_simulation(
    SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=0.6,
        c_gamma_att=0.6,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=0,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    SimulationRendererOriginal(
        pos_size=2 / 4,
        pos_color=(0, 255, 0),
        dir_len=5 / 4,
        dir_width=1 / 4,
        dir_color=(50, 255, 50),
    ),
    scale=2**-5,
)

In [37]:
run_interactive_simulation(
    SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=0.2,
        c_gamma_att=0.37,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=1,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    SimulationRendererOriginal(
        pos_size=2 / 8,
        pos_color=(0, 255, 0),
        dir_len=5 / 8,
        dir_width=1 / 8,
        dir_color=(50, 255, 50),
    ),
    scale=2**-5,
)


In [ ]:
run_batch_simulation(
    SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=0.2,
        c_gamma_att=0.37,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=1,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    rec := SimulationRecorderOriginal(skip_first_n=1000),
    steps=2000,
)
print(rec.dispersion)
print(rec.polarization)

8.019416335863994
0.5992334058296201


In [9]:
run_gamma_comparison_original(
    seed=0,
    att_vals=np.arange(0, 0.6, 0.05),
    ali_vals=np.arange(0, 1.2, 0.1),
    create_initial_state=lambda att, ali, seed: SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=ali,
        c_gamma_att=att,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=seed,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    runs_per_config=5,
    steps_per_run=500,
)

  0%|          | 0/720 [00:00<?, ?it/s]

array([[[6.46783703, 0.12124634],
        [6.33547889, 0.11913862],
        [6.28708662, 0.25102482],
        [6.26130564, 0.20658938],
        [6.20832289, 0.21386341],
        [6.22337639, 0.35631193],
        [6.29351085, 0.23711478],
        [6.02202216, 0.19214552],
        [6.07178721, 0.37253229],
        [6.05677805, 0.18747733],
        [5.87410082, 0.40014976],
        [6.03288039, 0.32372963]],

       [[6.31050846, 0.05407171],
        [6.25384064, 0.09853554],
        [6.21080633, 0.13582771],
        [6.26808438, 0.3144642 ],
        [6.01876645, 0.16929346],
        [6.14008898, 0.23269873],
        [6.15768393, 0.1795213 ],
        [6.05934864, 0.2104903 ],
        [6.27976257, 0.36275478],
        [6.22921714, 0.26502876],
        [5.85678128, 0.36037837],
        [5.8866821 , 0.16072003]],

       [[6.23963521, 0.10351961],
        [6.38115102, 0.12679834],
        [6.20133344, 0.21719   ],
        [6.09902507, 0.17355874],
        [6.1988571 , 0.25298139],
        [6